In [53]:
import pandas as pd
from bs4 import BeautifulSoup
import json
import requests
import re

In [32]:
keyword = '除濕機'
pages = 30
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}
urls = []
for page in range(1, pages):
    url = 'https://m.momoshop.com.tw/search.momo?_advFirst=N&_advCp=N&curPage={}&searchType=1&cateLevel=2&ent=k&searchKeyword={}&_advThreeHours=N&_isFuzzy=0&_imgSH=fourCardType'.format(page, keyword)
    print(url)
    resp = requests.get(url, headers=headers)
    if resp.status_code == 200:
        soup = BeautifulSoup(resp.text)
        for item in soup.select('li.goodsItemLi > a'):
            urls.append('https://m.momoshop.com.tw'+item['href'])
    urls = list(set(urls))
    print(len(urls))
#     break

https://m.momoshop.com.tw/search.momo?_advFirst=N&_advCp=N&curPage=1&searchType=1&cateLevel=2&ent=k&searchKeyword=除濕機&_advThreeHours=N&_isFuzzy=0&_imgSH=fourCardType
20
https://m.momoshop.com.tw/search.momo?_advFirst=N&_advCp=N&curPage=2&searchType=1&cateLevel=2&ent=k&searchKeyword=除濕機&_advThreeHours=N&_isFuzzy=0&_imgSH=fourCardType
40
https://m.momoshop.com.tw/search.momo?_advFirst=N&_advCp=N&curPage=3&searchType=1&cateLevel=2&ent=k&searchKeyword=除濕機&_advThreeHours=N&_isFuzzy=0&_imgSH=fourCardType
60
https://m.momoshop.com.tw/search.momo?_advFirst=N&_advCp=N&curPage=4&searchType=1&cateLevel=2&ent=k&searchKeyword=除濕機&_advThreeHours=N&_isFuzzy=0&_imgSH=fourCardType
80
https://m.momoshop.com.tw/search.momo?_advFirst=N&_advCp=N&curPage=5&searchType=1&cateLevel=2&ent=k&searchKeyword=除濕機&_advThreeHours=N&_isFuzzy=0&_imgSH=fourCardType
100
https://m.momoshop.com.tw/search.momo?_advFirst=N&_advCp=N&curPage=6&searchType=1&cateLevel=2&ent=k&searchKeyword=除濕機&_advThreeHours=N&_isFuzzy=0&_imgSH=f

In [168]:
df = []
for i, url in enumerate(urls):
    columns = []
    values = []
    
    resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(resp.text)
    # 標題
    title = soup.find('meta',{'property':'og:title'})['content']
    # 品牌
    brand = soup.find('meta',{'property':'product:brand'})['content']
    # 連結
    link = soup.find('meta',{'property':'og:url'})['content']
    # 原價
    try:
        price = re.sub(r'\r\n| ','',soup.find('del').text)
    except:
        price = ''
    # 特價
    amount = soup.find('meta',{'property':'product:price:amount'})['content']
    # 類型
    cate = ''.join([i.text for i in soup.findAll('article',{'class':'pathArea'})])
    cate = re.sub('\n|\xa0',' ',cate)
    # 描述
    try:
        desc = soup.find('div',{'class':'Area101'}).text
        desc = re.sub('\r|\n| ', '', desc)
    except:
        desc = ''
    
    print('==================  {}  =================='.format(i))    
    print(title)
    print(brand)
    print(link)
    print(amount)
    print(cate)
    
    columns += ['title', 'brand', 'link', 'price', 'amount', 'cate', 'desc']
    values += [title, brand, link, price, amount, cate, desc]

    # 規格
    for i in soup.select('div.attributesArea > table > tr'):
        try:
            column = i.find('th').text
            column = re.sub('\n|\r| ','',column)
            value = ''.join([j.text for j in i.findAll('li')])
            value = re.sub('\n|\r| ','',value)
            columns.append(column)
            values.append(value)
        except:
            pass
    ndf = pd.DataFrame(data=values, index=columns).T
    df.append(ndf)
df=pd.concat(df, ignore_index=True)

==================  0  ==================
【HITACHI 日立】6公升一級能效除濕機(RD-12BQ)
HITACHI 日立
http://m.momoshop.com.tw/goods.momo?i_code=5677582
6,356
  家電 >  除濕機 >  品牌總覽 >  HITACHI 日立  
==================  1  ==================
【Frigidaire 富及第★超值組合】節能補助政府退稅1200★新1級節能機種 26L清淨除濕機13-16坪(FDH-2631Y)
Frigidaire 富及第
http://m.momoshop.com.tw/goods.momo?i_code=7253971
9,540
  品牌旗艦 >  FRIGIDAIRE富及第 >  本月主打 >  刷momo卡最高回饋10%  
==================  2  ==================
送7-11商品卡100元【MITSUBISHI 三菱】10.5公升薄型大容量清淨除濕機(MJ-E105BJ)
MITSUBISHI 三菱
http://m.momoshop.com.tw/goods.momo?i_code=5251455
13,410
  家電 >  除濕機 >  品牌總覽 >  MITSUBISHI 三菱  
==================  3  ==================
送玻璃微波保鮮盒【HITACHI 日立】10公升一級能效清淨型除濕機(RD-200HH)
HITACHI 日立
http://m.momoshop.com.tw/goods.momo?i_code=7708291
13,006
  家電 >  除濕機 >  HITACHI日立 >  一級能效  
==================  4  ==================
超值組【SAMPO 聲寶】一級能效6公升空氣清淨除濕機(AD-WA712T+日本Siroca 塵蹣吸塵器SVC-358)
SAMPO 聲寶
http://m.momoshop.com.tw/goods.momo?i_code=7127104
6,999
  家電 >  除濕機 >  容量 / 公

==================  43  ==================
【MITSUBISHI 三菱】16公升一級能效日本原裝高效節能除濕機(MJ-E160HN)
MITSUBISHI 三菱
http://m.momoshop.com.tw/goods.momo?i_code=7461686
20,900
  家電 >  除濕機 >  本月主打 >  ★限時結帳87折↘  
==================  44  ==================
送玻璃微波保鮮盒【HITACHI 日立】14公升一級能效除濕機(RD-280HS/RD-280HG)
HITACHI 日立
http://m.momoshop.com.tw/goods.momo?i_code=7708627
13,671
  家電 >  除濕機 >  HITACHI日立 >  一級能效  
==================  45  ==================
【TATUNG 大同】8公升一級能效節能除濕機(TDH-168MB-NW)
TATUNG 大同
http://m.momoshop.com.tw/goods.momo?i_code=5789173
6,021
  家電 >  除濕機 >  品牌總覽 >  TATUNG 大同  
==================  46  ==================
【3M家電年中慶下單抽dyson吸塵器】3M 9.5L雙效空氣清淨除濕機FD-A90W可清淨/除濕/乾衣(加贈大同6人份電鍋)
3M
http://m.momoshop.com.tw/goods.momo?i_code=6105266
14,900
  家電 >  3M家電 >  除濕機 >  主機  
==================  47  ==================
【MITSUBISHI 三菱】25公升一級能效智慧變頻高效節能除濕機(MJ-EV250HM)
MITSUBISHI 三菱
http://m.momoshop.com.tw/goods.momo?i_code=5446384
24,273
  家電 >  除濕機 >  品牌總覽 >  MITSUBISHI 三菱  
==================  48  ==

==================  87  ==================
獨家超值組【歌林】智慧型電子式除濕機(KJ-MNR901)
Kolin 歌林
http://m.momoshop.com.tw/goods.momo?i_code=7468558
2,670
  家電 >  除濕機 >  除濕機專區 >  小空間使用  
==================  88  ==================
【HITACHI 日立】10公升一級能效除濕機(RD-200HS/RD-200HG)
HITACHI 日立
http://m.momoshop.com.tw/goods.momo?i_code=5505109
10,441
  家電 >  除濕機 >  品牌總覽 >  HITACHI 日立  
==================  89  ==================
【加碼贈專用濾網】3M 一級能效16公升雙效空氣清淨除溼機(旗艦新機FD-Y160L)
3M
http://m.momoshop.com.tw/goods.momo?i_code=7291727
19,990
  家電 >  3M家電 >  除濕機 >  主機  
==================  90  ==================
【MITSUBISHI 三菱】25公升一級能效智慧變頻高效節能除濕機(MJ-EV250HM)
MITSUBISHI 三菱
http://m.momoshop.com.tw/goods.momo?i_code=5428116
25,110
  家電 >  除濕機 >  除濕機專區 >  B式除濕機  
==================  91  ==================
【Opure 臻淨】D2 負離子除濕輪清淨除濕機
Opure 臻淨
http://m.momoshop.com.tw/goods.momo?i_code=4294302
6,309
  家電 >  除濕機 >  除濕機專區 >  清淨除濕  
==================  92  ==================
【MITSUBISHI 三菱】19.5公升一級能效日本原裝大容量強力型除濕機(MJ-E195HM)
MITSUBISHI

==================  131  ==================
【Panasonic 國際牌】6公升一級能效清淨除濕機(F-Y12EB)
Panasonic 國際牌
http://m.momoshop.com.tw/goods.momo?i_code=7646790
6,430
  家電 >  除濕機 >  本月主打 >  ★限時結帳87折↘  
==================  132  ==================
【HITACHI 日立】6公升一級能效除濕機(RD-12FQ)
HITACHI 日立
http://m.momoshop.com.tw/goods.momo?i_code=5471098
6,926
  家電 >  除濕機 >  品牌總覽 >  HITACHI 日立  
==================  133  ==================
【Panasonic 國際牌】14公升一級能效智慧節能清淨除濕機(F-Y28GX)
Panasonic 國際牌
http://m.momoshop.com.tw/goods.momo?i_code=7171020
14,220
  家電 >  除濕機 >  容量 / 公升數 >  10L-15L  
==================  134  ==================
【HITACHI 日立】10公升一級能效清淨型除濕機(RD-200HH)
HITACHI 日立
http://m.momoshop.com.tw/goods.momo?i_code=7278676
13,006
  家電 >  除濕機 >  HITACHI日立 >  一級能效  
==================  135  ==================
【SAMPO 聲寶】一級能效12公升全效淨化空氣清淨除濕機(AD-W724P)
SAMPO 聲寶
http://m.momoshop.com.tw/goods.momo?i_code=6884558
9,990
  家電 >  除濕機 >  品牌總覽 >  SAMPO 聲寶  
==================  136  ==================
【Panasonic 國際牌】8公升一級能效清淨除

==================  174  ==================
【BRI-RICH】智能雙芯片電子式空氣清淨除濕機(適用坪數8-10坪)
BRI-RICH
http://m.momoshop.com.tw/goods.momo?i_code=7735245
1,980
  家電 >  除濕機 >  容量 / 公升數 >  10L以下  
==================  175  ==================
【HITACHI 日立】18公升一級能效除濕機(RD-360HG)
HITACHI 日立
http://m.momoshop.com.tw/goods.momo?i_code=5606579
16,521
  家電 >  除濕機 >  品牌總覽 >  HITACHI 日立  
==================  176  ==================
【買兩台滿額加碼送千元mo幣★最高回饋1618mo幣】3M 一級能效16公升雙效空氣清淨除濕機(旗艦新機FD-Y160L)
3M
http://m.momoshop.com.tw/goods.momo?i_code=6482172
16,990
  家電 >  3M家電 >  本月主打 >  ★3M除濕機↘豪禮多選1  
==================  177  ==================
送玻璃微波保鮮盒【HITACHI 日立】14公升一級能效除濕機(RD-280HG)
HITACHI 日立
http://m.momoshop.com.tw/goods.momo?i_code=7708634
13,671
  家電 >  除濕機 >  HITACHI日立 >  一級能效  
==================  178  ==================
【HITACHI 日立】10公升一級能效清淨型除濕機(RD-200HH)
HITACHI 日立
http://m.momoshop.com.tw/goods.momo?i_code=7388553
13,690
  家電 >  除濕機 >  容量 / 公升數 >  10L-15L  
==================  179  ==================
送玻璃微波保鮮盒

...

==================  391  ==================
送玻璃微波保鮮盒【HITACHI 日立】12公升一級能效除濕機(RD-240HG)
HITACHI 日立
http://m.momoshop.com.tw/goods.momo?i_code=7708620
11,771
  家電 >  除濕機 >  HITACHI日立 >  一級能效  
==================  392  ==================
【HITACHI 日立】10公升一級能效除濕機-閃亮銀+【Coway】旗艦型空氣清淨機(RD-200HS + AP-1512HH)
HITACHI 日立
http://m.momoshop.com.tw/goods.momo?i_code=7729581
16,900
  家電 >  除濕機 >  HITACHI日立 >  一級能效  
==================  393  ==================
【Panasonic 國際牌】10公升一級能效ECONAVI空氣清淨除濕機(F-Y20FH)
Panasonic 國際牌
http://m.momoshop.com.tw/goods.momo?i_code=6234391
12,501
  家電 >  除濕機 >  容量 / 公升數 >  10L-15L  
==================  394  ==================
送7-11商品卡100元【MITSUBISHI 三菱】16公升一級能效日本原裝高效節能除濕機(MJ-E160HN)
MITSUBISHI 三菱
http://m.momoshop.com.tw/goods.momo?i_code=6279584
20,900
  家電 >  除濕機 >  品牌總覽 >  MITSUBISHI 三菱  
==================  395  ==================
【HITACHI 日立】14公升一級能效清淨型除濕機(RD-280HH)
HITACHI 日立
http://m.momoshop.com.tw/goods.momo?i_code=7328413
16,236
  家電 >  除濕機 >  容量 / 公升數 >  10L-15

==================  434  ==================
【HITACHI 日立】6公升一級能效除濕機(RD-12FQ)
HITACHI 日立
http://m.momoshop.com.tw/goods.momo?i_code=7164974
6,926
  家電 >  除濕機 >  除濕機專區 >  全系列  
==================  435  ==================
送玻璃微波保鮮盒【HITACHI 日立】16公升一級能效清淨型除濕機(RD-320HH)
HITACHI 日立
http://m.momoshop.com.tw/goods.momo?i_code=7710851
18,326
  家電 >  除濕機 >  HITACHI日立 >  含清淨除濕  
==================  436  ==================
送聲寶14吋DC風扇【Whirlpool惠而浦】26.5L節能除濕機 WDEE60AW(二級能效貨物稅減免$1200)
Whirlpool 惠而浦
http://m.momoshop.com.tw/goods.momo?i_code=6542372
12,500
  家電 >  除濕機 >  除濕機專區 >  B式除濕機  
==================  437  ==================
【Healthlead】日式迷你防潮除濕機(白EPI-608C)
Healthlead
http://m.momoshop.com.tw/goods.momo?i_code=1385543
2,380
  家電 >  除濕機 >  除濕機專區 >  電子式  
==================  438  ==================
【HITACHI 日立】10公升一級能效除濕機(RD-20FQ)
HITACHI 日立
http://m.momoshop.com.tw/goods.momo?i_code=7188646
9,120
  家電 >  除濕機 >  除濕機專區 >  全系列  
==================  439  ==================
獨家贈快煮壺-【HERAN 禾聯】2019年新一級能效11公

==================  478  ==================
【Panasonic 國際牌】16公升一級能效智慧節能清淨除濕機(F-Y32GX)
Panasonic 國際牌
http://m.momoshop.com.tw/goods.momo?i_code=7664026
15,561
  家電 >  除濕機 >  Panasonic國際牌 >  一級能效  
==================  479  ==================
【HealthLife】環保節能迷你防潮除濕機  HL-610(除濕機)

http://m.momoshop.com.tw/goods.momo?i_code=3596931
2,280
  家電 >  除濕機 >  容量 / 公升數 >  10L以下  
==================  480  ==================
送玻璃微波保鮮盒【HITACHI 日立】10公升一級能效除濕機(RD-200HS)
HITACHI 日立
http://m.momoshop.com.tw/goods.momo?i_code=7708604
10,441
  家電 >  除濕機 >  HITACHI日立 >  一級能效  
==================  481  ==================
【HITACHI 日立】14公升一級能效清淨型除濕機(RD-280HH)
HITACHI 日立
http://m.momoshop.com.tw/goods.momo?i_code=7388555
17,090
  家電 >  除濕機 >  容量 / 公升數 >  10L-15L  
==================  482  ==================
【NEW WIDETECH 威技】18公升除濕機(WDH-1150DD-18R)
NEW WIDETECH 威技
http://m.momoshop.com.tw/goods.momo?i_code=6959141
5,990
  家電 >  除濕機 >  品牌總覽 >  威技電器  
==================  483  ==================
【HITACHI 日立】14公升一級能效除

==================  522  ==================
【HITACHI 日立】10公升一級能效除濕機(RD-20FQ/RD-20FR)
HITACHI 日立
http://m.momoshop.com.tw/goods.momo?i_code=6492962
9,120
  家電 >  除濕機 >  品牌總覽 >  HITACHI 日立  
==================  523  ==================
送玻璃微波保鮮盒【HITACHI 日立】12公升一級能效清淨型除濕機(RD-240HH)
HITACHI 日立
http://m.momoshop.com.tw/goods.momo?i_code=7708615
14,336
  家電 >  除濕機 >  HITACHI日立 >  一級能效  
==================  524  ==================
【CHIMEI 奇美】12L時尚美型節能除濕機 RH-12E0RM+2in1多功能無線吸塵器 VC-HC4LS0(超值組合)
CHIMEI 奇美
http://m.momoshop.com.tw/goods.momo?i_code=7127124
11,990
  家電 >  除濕機 >  品牌總覽 >  CHIMEI 奇美  
==================  525  ==================
【BRI-RICH】智能雙芯片電子式空氣清淨除濕機(適用坪數8-10坪)
BRI-RICH
http://m.momoshop.com.tw/goods.momo?i_code=6190805
1,980
  家電 >  除濕機 >  除濕機專區 >  電子式  
==================  526  ==================
【HITACHI 日立】8公升一級能效除濕機(RD-16FQ/RD-16FR)
HITACHI 日立
http://m.momoshop.com.tw/goods.momo?i_code=5498747
8,161
  家電 >  除濕機 >  品牌總覽 >  HITACHI 日立  
==================  527  ==================

==================  566  ==================
【Panasonic 國際牌】16公升一級能效清淨除濕機(F-Y32EH)
Panasonic 國際牌
http://m.momoshop.com.tw/goods.momo?i_code=5230706
18,234
  家電 >  除濕機 >  品牌總覽 >  Panasonic 國際牌  
==================  567  ==================
【6/21前滿3千送300/滿6千送618mo幣】3M 9.5L雙效空氣清淨除濕機FD-A90W可清淨/除濕/乾衣(贈Siroca吸塵器)
3M
http://m.momoshop.com.tw/goods.momo?i_code=5512056
13,990
  家電 >  3M家電 >  除濕機 >  主機  
==================  568  ==================
【MITSUBISHI 三菱】16公升一級能效日本原裝高效節能除濕機(MJ-E160HN)
MITSUBISHI 三菱
http://m.momoshop.com.tw/goods.momo?i_code=6574471
20,900
  家電 >  除濕機 >  品牌總覽 >  MITSUBISHI 三菱  
==================  569  ==================
【Panasonic 國際牌】11公升一級能效ECONAVI空氣清淨除濕機(F-Y22EN)
Panasonic 國際牌
http://m.momoshop.com.tw/goods.momo?i_code=5201418
8,991
  家電 >  除濕機 >  除濕機專區 >  全系列  
==================  570  ==================
【福利品Kolin 歌林】超大容量智慧節能 24公升除濕機(KJ-A251B)
Kolin 歌林
http://m.momoshop.com.tw/goods.momo?i_code=7450780
8,963
  家電 >  除濕機 >  容量 / 公升數 >  16L以上  
==================  571  =

In [169]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 580 entries, 0 to 579
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   580 non-null    object
 1   brand   580 non-null    object
 2   link    580 non-null    object
 3   price   580 non-null    object
 4   amount  580 non-null    object
 5   cate    580 non-null    object
 6   desc    580 non-null    object
 7   品牌名稱    557 non-null    object
 8   類型      580 non-null    object
 9   功效      575 non-null    object
 10  容量      536 non-null    object
 11  適用於     575 non-null    object
 12  功能      575 non-null    object
 13  效能      567 non-null    object
 14  保固期     488 non-null    object
 15  配件      20 non-null     object
dtypes: object(16)
memory usage: 72.6+ KB


In [170]:
df.to_excel('./MOMO.xlsx')

In [171]:
df

,title,brand,link,price,amount,cate,desc,品牌名稱,類型,功效,容量,適用於,功能,效能,保固期,配件
0,【HITACHI 日立】6公升一級能效除濕機(RD-12BQ),HITACHI 日立,http://m.momoshop.com.tw/goods.momo?i_code=567...,"6,690","6,356",家電 > 除濕機 > 品牌總覽 > HITACHI 日立,商品規格：型號：RD-12BR/BQ顏色：BR(玫瑰金)、BQ(閃亮銀)適用室溫範圍：5℃~...,HITACHI日立,一般濾網一般插頭式,2L~9L,10L以下,4坪~6坪,滿水提醒,1級能效,3年,NaN
1,【Frigidaire 富及第★超值組合】節能補助政府退稅1200★新1級節能機種 26L清...,Frigidaire 富及第,http://m.momoshop.com.tw/goods.momo?i_code=725...,"10,600","9,540",品牌旗艦 > FRIGIDAIRE富及第 > 本月主打 > 刷momo卡最高回饋1...,型號：FDH-2631Y商品尺寸(高*寬*深)：590x346x242mm重量(kg)：14...,Frigidaire富及第,一般濾網一般插頭式,21L~30L,10L以下,1坪~3坪4坪~6坪7坪~9坪10坪~12坪13坪~15坪16坪~19坪,連續除濕,1級能效,1年,NaN
2,送7-11商品卡100元【MITSUBISHI 三菱】10.5公升薄型大容量清淨除濕機(MJ...,MITSUBISHI 三菱,http://m.momoshop.com.tw/goods.momo?i_code=525...,"14,900","13,410",家電 > 除濕機 > 品牌總覽 > MITSUBISHI 三菱,除濕方式：壓縮機式除濕能力：10.5L/日適用室溫範圍：1℃?35℃除濕坪數：7/13坪耗電...,MITSUBISHI三菱,一般濾網一般插頭式,10L~15L,10L以下,1坪~3坪4坪~6坪7坪~9坪,滿水提醒,4級能效,1年,NaN
3,送玻璃微波保鮮盒【HITACHI 日立】10公升一級能效清淨型除濕機(RD-200HH),HITACHI 日立,http://m.momoshop.com.tw/goods.momo?i_code=770...,"13,690","13,006",家電 > 除濕機 > HITACHI日立 > 一級能效,【商品規格】\t\t\t\t\t型號：RD-200HH(天晴藍)除濕能力：10公升/日額定消...,HITACHI日立,HEPA濾網一般插頭式,10L~15L,10L以下,1坪~3坪4坪~6坪7坪~9坪10坪~12坪13坪~15坪,滿水提醒,1級能效,NaN,NaN
4,超值組【SAMPO 聲寶】一級能效6公升空氣清淨除濕機(AD-WA712T+日本Siroca...,SAMPO 聲寶,http://m.momoshop.com.tw/goods.momo?i_code=712...,"7,870","6,999",家電 > 除濕機 > 容量 / 公升數 > 10L以下,聲寶除濕機◆型號：AD-WA712T◆消耗功率：114W◆額定電壓/額定頻率：110V/60...,SAMPO聲寶,一般濾網一般插頭式,2L~9L,10L以下,1坪~3坪4坪~6坪,預約功能連續除濕,1級能效,3年,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575,送奇美小偏心果汁機【Whirlpool惠而浦】6L節能除濕機 WDEM12W,Whirlpool 惠而浦,http://m.momoshop.com.tw/goods.momo?i_code=768...,"5,990","4,690",家電 > 除濕機 > 本月主打 > ★惠而浦除濕機指定品送豪禮,【Whirlpool惠而浦】6L節能除濕機(WDEM12W)◆型號：WDEM12W◆色彩：典...,Whirlpool惠而浦,HEPA濾網一般插頭式,2L~9L,10L以下,1坪~3坪4坪~6坪7坪~9坪,滿水提醒預約功能連續除濕,4級能效,1年,NaN
576,【HITACHI 日立】8公升一級能效除濕機(RD-16FQ/RD-16FR),HITACHI 日立,http://m.momoshop.com.tw/goods.momo?i_code=649...,"8,590","8,161",家電 > 除濕機 > 品牌總覽 > HITACHI 日立,"商品型號：RD-16FQ(閃亮銀),RD-16FR(玫瑰金)除濕能力：8公升/日額定消耗電功...",HITACHI日立,一般濾網一般插頭式,2L~9L,10L以下,1坪~3坪,自動除霜,1級能效,3年,NaN
577,【HERAN 禾聯★滿額登記送MO幣】貨物稅減免$900★獨家下殺★新一級能效11公升除濕機...,HERAN 禾聯,http://m.momoshop.com.tw/goods.momo?i_code=703...,"7,990","4,599",家電 > 除濕機 > 品牌總覽 > HERAN 禾聯,品牌名稱：HERAN禾聯型號：HDH-22YL010輸入電源：110V/60Hz額定消耗功率...,HERAN禾聯,一般濾網一般插頭式,10L~15L,11-20L,7坪~9坪10坪~12坪,滿水提醒預約功能自動除霜連續除濕,1級能效,1年,NaN
578,【HITACHI 日立】6公升一級能效除濕機(RD-12BQ/RD-12BR),HITACHI 日立,http://m.momoshop.com.tw/goods.momo?i_code=649...,"6,690","6,356",家電 > 除濕機 > 品牌總覽 > HITACHI 日立,商品型號：RD-12BQ(閃亮銀)/RD-12BR(玫瑰金)除濕能力：6公升/日額定消耗電功...,HITACHI日立,一般濾網一般插頭式,2L~9L,10L以下,1坪~3坪,自動除霜,1級能效,3年,NaN
